In [7]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Projects/Perturbed-Equilibrium/JPEC.worktrees/vacuum_julia`


In [11]:
function arrays!(
    mth::Int,
    dth::Float64,
    lmin::Vector{Int},
    lmax::Vector{Int},
    qa1::Float64,
    n::Int,
    delfac::Float64,
    delta::Vector{Float64},
    xpla::Vector{Float64},
    zpla::Vector{Float64},
    grri::Array{Float64,2},
    farwal::Bool
)
    # Sizes
    mth1 = mth + 1
    mth12 = 2 * mth
    jmax1 = lmax[1] - lmin[1] + 1
    q = qa1
    nq = n * q

    # Compute geometry bounds
    xmin = minimum(xpla)
    xmax = maximum(xpla)
    zmin = minimum(zpla)
    zmax = maximum(zpla)

    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    delx = plrad * delfac
    delz = plrad * delfac

    println("  delx, dely = ", delx, " ", delz)

    # Poloidal angle grid
    the = [(i-1) * dth for i in 1:mth1]

    # Wall shape (placeholder: straight line wall matching plasma)
    xwal = copy(xpla[1:mth1])
    zwal = copy(zpla[1:mth1])

    # Derivatives (simple finite difference instead of spline)
    xwalp = [ (xwal[mod1(i+1,mth)] - xwal[i]) / dth for i in 1:mth ]
    zwalp = [ (zwal[mod1(i+1,mth)] - zwal[i]) / dth for i in 1:mth ]
    push!(xwalp, xwalp[1])
    push!(zwalp, zwalp[1])

    # Allocate arrays
    cplar = zeros(Float64, mth1, jmax1)
    cplai = zeros(Float64, mth1, jmax1)
    cwallr = zeros(Float64, mth1, jmax1)
    cwalli = zeros(Float64, mth1, jmax1)
    cnqd = zeros(Float64, mth1)
    snqd = zeros(Float64, mth1)
    sinlt = zeros(Float64, mth1, jmax1)
    coslt = zeros(Float64, mth1, jmax1)
    snlth = zeros(Float64, mth1, jmax1)
    cslth = zeros(Float64, mth1, jmax1)

    # Plasma and wall coefficients
    for l1 in 1:jmax1
        for i in 1:mth
            cplar[i,l1] = grri[i,l1]
            cplai[i,l1] = grri[i,jmax1+l1]
            if !farwal
                cwallr[i,l1] = grri[mth+i,l1]
                cwalli[i,l1] = grri[mth+i,jmax1+l1]
            end
        end
        cplar[mth1,l1] = cplar[1,l1]
        cplai[mth1,l1] = cplai[1,l1]
        if !farwal
            cwallr[mth1,l1] = cwallr[1,l1]
            cwalli[mth1,l1] = cwalli[1,l1]
        end
    end

    # Trigonometric basis arrays
    for is in 1:mth1
        theta = (is-1) * dth
        znqd = nq * delta[is]
        cnqd[is] = cos(znqd)
        snqd[is] = sin(znqd)
        for l1 in 1:jmax1
            ll = lmin[1] - 1 + l1
            elth = ll * theta
            elthnq = ll * theta + znqd
            sinlt[is,l1] = sin(elth)
            coslt[is,l1] = cos(elth)
            snlth[is,l1] = sin(elthnq)
            cslth[is,l1] = cos(elthnq)
        end
    end

    return (
        delx, delz, the, xwal, zwal, xwalp, zwalp,
        cplar, cplai, cwallr, cwalli,
        cnqd, snqd, sinlt, coslt, snlth, cslth
    )
end


arrays! (generic function with 1 method)

In [12]:
# ----------------------------------------------------
# Test harness
function test_arrays()
    mth = 8
    mth1 = mth + 1
    dth = 2π / mth
    lmin = [0]
    lmax = [3]
    qa1 = 1.0
    n = 1
    delfac = 0.1
    farwal = false

    # Make sure these arrays have length mth1 to avoid bounds error
    the = [(i-1) * dth for i in 1:mth1]
    xpla = cos.(the)   # length 9
    zpla = sin.(the)   # length 9
    delta = zeros(mth1)  # length 9

    jmax1 = lmax[1] - lmin[1] + 1

    # Use Float64 grri, sized (2*mth, 2*jmax1)
    grri = Float64.( [i + j for i in 1:2*mth, j in 1:2*jmax1] )

    # Call arrays!
    return arrays!(mth, dth, lmin, lmax, qa1, n, delfac, delta, xpla, zpla, grri, farwal)
end

# Run test and print some outputs
result = test_arrays()

println("delx = ", result[1], "  delz = ", result[2])
println("thetas (first 5): ", result[3][1:5])
println("xwal (first 5): ", result[4][1:5])
println("zwal (first 5): ", result[5][1:5])
println("xwalp (first 5): ", result[6][1:5])
println("zwalp (first 5): ", result[7][1:5])
println("cplar (3x3 block):")
println(result[8][1:3, 1:3])
println("cplai (3x3 block):")
println(result[9][1:3, 1:3])

  delx, dely = 0.1 0.1
delx = 0.1  delz = 0.1
thetas (first 5): [0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793]
xwal (first 5): [1.0, 0.7071067811865476, 6.123233995736766e-17, -0.7071067811865475, -1.0]
zwal (first 5): [0.0, 0.7071067811865475, 1.0, 0.7071067811865476, 1.2246467991473532e-16]
xwalp (first 5): [-0.37292322857805654, -0.9003163161571061, -0.9003163161571062, -0.3729232285780567, 0.37292322857805643]
zwalp (first 5): [0.9003163161571061, 0.3729232285780567, -0.37292322857805654, -0.9003163161571061, -0.9003163161571062]
cplar (3x3 block):
[2.0 3.0 4.0; 3.0 4.0 5.0; 4.0 5.0 6.0]
cplai (3x3 block):
[6.0 7.0 8.0; 7.0 8.0 9.0; 8.0 9.0 10.0]
